In [13]:
import datasets
from collections import Counter
import string
from gensim.utils import tokenize
from torch.utils.data import Dataset,DataLoader
from torch.nn import Module 
from torch import nn 
import torch


In [2]:
newsdata=datasets.load_dataset('ag_news')

In [3]:
newsdata['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [4]:
#Создаём словарь количества вхождений слов
words=['<unk>','<bos>','<eos>','<pad>']
#str.maketrans('','',string.punctuation)-возврящяет словарь для замены
#.translate ждёт словарь слов для замены
for sent in newsdata['train']['text']:
    proced_sent=sent.lower().translate(
        str.maketrans('','',string.punctuation)
    )
    for word in tokenize(proced_sent):
        words.append(word)
        

vocab = set(['<unk>','<bos>','<eos>','<pad>'])#Токеный неизвестногого слова, начала,конца последовательности и токен пустного пропуска для батчей


treshold=25 #Порог для включения в словарь word2ind
words=Counter(words)

for word,cnt in words.items():
    if cnt>treshold:
        vocab.add(word)
print(f'Размер словаря {len(vocab)}')
        
   

Размер словаря 11505


In [5]:
word2ind={i:char for char,i in enumerate(vocab)}
ind2word={char:i for char,i in enumerate(vocab)}

In [6]:
class WordDataset(Dataset):
    def __init__(self,data):  
        super(WordDataset,self).__init__()
        self.data=data
        self.unk=word2ind['<unk>']
        self.bos=word2ind['<bos>']
        self.eos=word2ind['<eos>']
        self.pad=word2ind['<pad>']        
    def __getitem__(self,idx:int):
        #получаем оригинальные данные
        #print(self.data)
        sent=self.data['text'][idx]
        #print(sent)
        label=self.data['label'][idx]
        proc_sent=sent.lower().translate(
            str.maketrans('','',string.punctuation)

        )
        #tokenized_sent=tokenize(proc_sent)
        tokenized_sent=[self.bos]
        tokenized_sent+=[
            word2ind.get(word,self.unk) for word in tokenize(proc_sent)
        ]
        tokenized_sent+=[self.eos]

        sample={
            'text':tokenized_sent,
            'label':label
        }
        return sample
    def __len__(self):
        return len(self.data)

In [92]:
train_dataset=WordDataset(newsdata['train'])
eval_dataset=WordDataset(newsdata['test'][0:100])

In [17]:
def collate_fn_with_padding(
    input_batch: list[list[int]], pad_id=word2ind['<pad>'], max_len=256) -> torch.Tensor:
    seq_lens = [len(x['text']) for x in input_batch]
    max_seq_len = min(max(seq_lens), max_len)

    new_batch = []
    for sequence in input_batch:
        sequence['text'] = sequence['text'][:max_seq_len]
        for _ in range(max_seq_len - len(sequence['text'])):
            sequence['text'].append(pad_id)

        new_batch.append(sequence['text'])

    sequences = torch.LongTensor(new_batch)
    labels = torch.LongTensor([x['label'] for x in input_batch])

    new_batch = {
        'input_ids': sequences,
        'label': labels
    }

    return new_batch

In [29]:
def make_batch(data,max_len=256,pad_id=word2ind['<pad>']):
    lenghts=[len(sent['text']) for sent in data]
    max_len=min(max_len,max(lenghts))
    new_batch=[]
    for sent in data:
        sent['text']=sent['text'][:max_len]
        for i in range(max_len-len(sent['text'])):
            sent['text'].append(pad_id)
        new_batch.append(sent['text'])

    new_batch=torch.LongTensor(new_batch)

    new_pair={
        'text':torch.LongTensor(new_batch),
        'label':torch.LongTensor([x['label'] for x in data])
    }
    return new_pair

    

In [32]:
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True,collate_fn=make_batch)
eval_dataloader=

In [72]:
class rnn_net(Module):
    def __init__(self,inp_size,hidden_size,out_size,vocab_size):
        super(rnn_net,self).__init__()

        self.emb=nn.Embedding(vocab_size,hidden_size)
        self.rnn=nn.RNN(hidden_size,hidden_size,batch_first=True)#!!!!!!!1
        self.fin_lin=nn.Linear(hidden_size,out_size)
        self.tahn=nn.Tanh()
    def forward(self,x):
        x=self.emb(x)
        x,_=self.rnn(x)
        x=self.tahn(x)
    

        #агрегация эмбрендингов
        x=x.mean(dim=1)

        
        out=self.fin_lin(x)

        return x
        

In [73]:
model=rnn_net(256,256,4,len(vocab))
loss_func=nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer=torch.optim.Adam(model.parameters())

In [78]:
def training(model,dataloader,loss_func,optimizer):
    for batch in dataloader:

        optimizer.zero_grad()

        
        pred=model(batch['text'])
        
        
        loss=loss_func(pred,batch['label'])
        loss_item=loss.item()
        loss.backward()
        optimizer.step()

        print(loss_item)

        

        

In [80]:
def get_accuracy(model,dataloader):
    with torch.no_grad():
        pred_labels=[model(x['text']) for x in dataloader]
        real_labels=[x['label'] for x in dataloader]
    pred_labels=torch.cat(pred_labels)
    real_labels=torch.cat(real_labels)

    accuracy=(pred_labels==real_labels).float.mean()
    return accuracy
        
        

In [79]:
training(model=model,dataloader=train_dataloader,loss_func=loss_func,optimizer=optimizer)

5.756989479064941
5.437158107757568
5.112077713012695
5.06309175491333
5.140541076660156
5.1082987785339355
5.062184810638428
4.9788899421691895
4.875540256500244
4.767197132110596


KeyboardInterrupt: 